In [1]:
# nltk.download('punkt') #uncomment if running on new machine

In [1]:
# Importing the libraries
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gensim
from gensim.test.utils import datapath
from gensim.models import Word2Vec
import os
import nltk
from nltk import word_tokenize
from nltk import sent_tokenize

# Helpful variables
EXT_DATA_FOLDER = "C:\\Users\\Admin\\Projects\\thesis\\data\\"
EXT_DATA_FOLDER2 = "B:\\Datasets\\"

ANALYSIS_SAMPLES = os.path.join(EXT_DATA_FOLDER, "Credibility_Analysis_Samples\\September_25\\")
dataset_columns = ['Identifier', 'Type', 'Category', 'URL', 'Cat1', 'Cat2', 'Cat3', 'Cat4', 'Cat5',
 'Cat6', 'Cat7', 'Score', 'First date_time', 'Tweets', 'Likes', 'Retweets',
 'Potential exposure', 'HTML', 'TEXT']
criterias = ["Cat1", "Cat2", "Cat3", "Cat4", "Cat5", "Cat6", "Cat7"]


C:\Users\Admin\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Low Credibility Score Detector

1) Create a dataset that separates he articles as having low credibility and not low credibility where low credibility is defined to be articles with a credibility score of less than 3

2.a) Apply the ensemble of models to classify an article for each category and then use the resulting score to determine whether an article's credibility is low or not

2.b) Apply a new model that has been trained to solely just determine whether an article's credibility is low or not

3) ???

4) Profit

In [2]:
from collections import defaultdict

labelled_articles = pd.read_excel("dataset5.xlsx")
labelled_articles = labelled_articles.dropna(subset=['TEXT'])
labelled_articles = labelled_articles[pd.to_numeric(labelled_articles['Cat1'], errors='coerce').notnull()]


In [11]:
labelled_articles['Low Credibility'] = np.where(labelled_articles['Score'] < 3, 1, 0)

### Training and saving model


In [ ]:
art_text_sent = np.array([sent_tokenize(article.split("TITLE: ")[1].replace("TEXT: ","").strip(" ")) for article in labelled_articles["TEXT"]])
art_text_word = np.array([word_tokenize(article.split("TITLE: ")[1].replace("TEXT: ","").strip(" ")) for article in labelled_articles["TEXT"]])
art_text_sent_word = np.array([[word_tokenize(sent) for sent in article] for article in art_text_sent])

In [25]:
excerpt = """Thousands of parents describe their children as ‘fine’ one day, then their children suddenly develop autism (a neurological regression) after vaccines.
             What does the science say? It agrees…"""

sentences = np.array(sent_tokenize(excerpt))
print(sentences)
final = np.array([[word_tokenize(sent) for sent in sentences]])
print(final)

['Thousands of parents describe their children as ‘fine’ one day, then their children suddenly develop autism (a neurological regression) after vaccines.'
 'What does the science say?' 'It agrees…']
[[list(['Thousands', 'of', 'parents', 'describe', 'their', 'children', 'as', '‘', 'fine', '’', 'one', 'day', ',', 'then', 'their', 'children', 'suddenly', 'develop', 'autism', '(', 'a', 'neurological', 'regression', ')', 'after', 'vaccines', '.'])
  list(['What', 'does', 'the', 'science', 'say', '?'])
  list(['It', 'agrees…'])]]


### Example Code

In [ ]:
for criteria in criterias:
    print("Training Classifier for " + criteria + ":")
    start_time = timeit.default_timer()

    X_train, X_test, y_train, y_test = train_test_split(list(labelled_articles["TEXT"]), list(labelled_articles[criteria]), test_size=int(20))

    svm_clf = Pipeline([('vect', CountVectorizer(max_features=10)),
                         ('tfidf', TfidfTransformer()),
                         ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                               alpha=1e-3, random_state=69,
                                               max_iter=5, tol=None)),
    ])
    svm_clf.fit(X_train, y_train)
    svm_predicted = list(svm_clf.predict(X_test))
    svm_cv_scores = cross_val_score(svm_clf, X_train, y_train, cv=10, scoring='f1_micro')

    print("SVM Average micro f1-score: %0.2f (+/- %0.2f)" % (svm_cv_scores.mean(), svm_cv_scores.std()))
    joblib.dump(svm_clf, 'models/svm_clf_tfidf_stop_' + criteria + '.joblib')

### Single Model Approach
SVM + TF-IDF with stopwords removed

In [ ]:
def build_train(article_text, labels, criteria):
    print("Training Classifier for " + criteria + ":")
    start_time = timeit.default_timer()

    X_train, X_test, y_train, y_test = train_test_split(article_text, labels, test_size=int(20))

    svm_clf = Pipeline([('vect', CountVectorizer(max_features=10)),
                         ('tfidf', TfidfTransformer()),
                         ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                               alpha=1e-3, random_state=69,
                                               max_iter=5, tol=None)),
    ])
    svm_clf.fit(X_train, y_train)
    svm_predicted = list(svm_clf.predict(X_test))
    svm_cv_scores = cross_val_score(svm_clf, X_train, y_train, cv=10, scoring='f1_micro')

    print("SVM Average micro f1-score: %0.2f (+/- %0.2f)" % (svm_cv_scores.mean(), svm_cv_scores.std()))
    joblib.dump(svm_clf, 'models/svm_clf_tfidf_stop_' + criteria + '.joblib')
    
    